# Lab 13

On Wednesday, we focused on the process of supervised learning, specifically spending time with the training and testing phases. Today, we examine the limitations of how a single iteration of train and test, and introduce the idea of cross-validation for addressing some of these weaknesses.

0. Detail limitations of "vanilla" train/test paradigm
1. Define and apply cross-validation

**_Note_** - Since I am out of town, I have added time markers for how long I _think_ each piece takes. You are not limited to these time markers; I am only attempting to provide guidence. 

### Imports for Today

Today we are working with the data from last time. You can either import the data from the Lab 12 directory directly or by copying it into this directory. 

We begin as usual, importing the packages and data that we need. Still do **not** plot the data. 

In [ ]:
## Import block
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import numpy as np
from numpy import linalg as LA
import pandas as pd

import random 

from sklearn import linear_model

In [ ]:
## Functions for later use

def compute_mse(truth_vec, predict_vec):
    return np.mean((truth_vec - predict_vec)**2)


In [ ]:
# For function testing 




In [ ]:
## Import Data

mystery_data = pd.read_csv("lab12data.csv", sep = ",")
mystery_np = np.genfromtxt("lab12data.csv", delimiter=',', skip_header=1)


# DO NOT PLOT ANYTHING! 

In [ ]:
# General x-values for fitting and plotting
x = np.linspace(np.min(train_data[:,1])-0.1, np.max(train_data[:,1])+0.1, 1000)
x = np.array([x]).T

x_mat = np.hstack([x,x**2,x**3,x**4,x**5,x**6,x**7,x**8,x**9,x**10])

### Training and Testing: What is the "right" approach?

In the last lab, we started to play with the intuition behind each of the train and test phases. You generated images of the training and testing errors for various proportions for the train and test sets. 

#### Warm-Up Part 1 (10 minutes) 

Let's begin with a quick activity. Let's divide the data into 5 equal parts: 
* Set A - Rows 0:59
* Set B - Rows 60:119
* Set C - Rows 120:179
* Set D - Rows 180:239
* Set E - Rows 240:299

Assume the data follows a 2-degree polynomial, compute the training and testing errors where each set acts as the training set with the other four combined as the testing set. Plot the training and test errors. What do you notice? 

In [1]:
# Dividing the data into the five sets


In [ ]:
# Computing train and test errors for where
#    each set acts as the training data 
#    for a 2-degree polynomial

In [ ]:
# Plot the training and testing errors in order 
# of the Sets A, B, C, D, E

**Question:** Given these plots, what statements/questions/concerns do you have about how we split data into train and test? 

(Spot for your notes)

#### Warm-Up Part 2 (7 minutes)

Repeat part 1, but for a 3-degree polynomial and for a 4-degree polynomial. I would strongly recommend that you copy and paste the code below, instead of just rerunning the above blocks. Again, below each plot, answer the question from part 1 (copied below). 

In [1]:
# Dividing the data into the five sets


In [ ]:
# Computing train and test errors for where
#    each set acts as the training data 
#    for a 3-degree polynomial

In [ ]:
# Plot the training and testing errors in order 
# of the Sets A, B, C, D, E

**Question:** Given these plots, what statements/questions/concerns do you have about how we split data into train and test? 

(Spot for your notes)

In [1]:
# Dividing the data into the five sets


In [ ]:
# Computing train and test errors for where
#    each set acts as the training data 
#    for a 4-degree polynomial

In [ ]:
# Plot the training and testing errors in order 
# of the Sets A, B, C, D, E

**Question:** Given these plots, what statements/questions/concerns do you have about how we split data into train and test? 

(Spot for your notes)

#### Warm-up Part 3 (3 minutes)
Now, make one plot with three lines for the testing errors in order of the Sets A, B, C, D, E. 

In [ ]:
# Three testing error lines on one plot

Before continuing answer a few questions:
**Q1:** What surprises you about this plot?   
**A1:** 

**Q2:** Which training set is the right one? Why?     
**A2:** 

**Q3:** Of these three polynomial degrees (2,3,4), which is the best one for this data? Why?     
**A3:** 

## Limitations for the train and test paradigm

It can be very surprising that the training and testing errors can vary so wildly depending on what data is in the training set and what data is in the testing set. This is because of the _variance_ that is inherent in any model about data, due to the variations in data. Recalling that the goal of any machine learning algorithm is to uncover some underlying truth about our data, in seeking this truth, we can't rely on having the "right" training data. Instead, we need to have methods that are robust to variations in our data; in other words, we want to feel confident that our discoveries are good regardless of the data that we train on. 

Our goal with the `mystery_data` is to uncover the polynomial relationship between `mysty` and `myst`. In Lab 12, we divided our data into train and test. Then for degrees between 1 and 10, we used the training data to find a polynomial relationship, and used the testing set to see how this found relationship might generalize to new data. We noticed a few things:
* The training error went down as we increased the degree of the polynomial
* The test error dropped for a bit and then increased, as we increased the degree of the polynomial

In Lab 12, we also experimented with the proportion of the data that we classified as the training set. Then in the first part of this lab, we investigated which part of the data is the "right" training data. With the goal of trying to uncover the "right" polynomial relationship for `mysty` and `myst`, we have felt less than satisfied with our choices. (Note, this was intentional.) 

With the goal of figuring out the "right" polynomial for `mysty` and `myst`, we have to determine the correct degree (or shape) for our polynomial model. 

_Side Question:_ what is the degree of the polynomial: parameter or hyperparameter? 

If we rely solely on training error or choose the wrong training set, we risk overfitting to our training data. We need something that will punish us for adding degrees for the sake of adding degrees, but also reward us for adding the right number of degrees. So what are we to do? 

## Cross-Validation

**Cross-Validation** is an iterative version of the train/test paradigm where each data point has a turn in the training set. There are a few varieties of _cross-validation_ (also colloquially called cross-val), but they all have the same basic structure:

0. Choose the model that you want to validate (ie. see how it would generalize to new data)
1. Divide your data into $k$ equal pieces
2. Do the following $k$ times: 
   * Select the $i^{\textrm{th}}$ piece of the dataset to be the training set
   * The rest of the pieces are your test set
   * Fit your model on the training set
   * Compute the test error (ie. MSE) and call this the $i^{\textrm{th}}$ test error
   * Add the $i^{\textrm{th}}$ test error to a list of test errors
3. Compute the average of the test errors

When $k$ does not equal the number of data points, we call thsi $k$-fold cross-validation. The _folds_ are 

In [ ]:
random.seed(2019)

train_inds = random.sample(list(range(mystery_np.shape[0])),15)
train_data = mystery_np[train_inds,:]

test_inds = list(set(range(mystery_np.shape[0])).difference(set(train_inds)))
test_data = mystery_np[test_inds,:]

Plot these points and make a guess of how `mysty` is constructed from `myst`.

In [ ]:
plt.scatter(train_data[:,1],train_data[:,0])

### Testing all the possible polynomials

We have been told that `mysty` and `myst` have a polynomial relationship and our broad goal is to uncover the true polynomial relationship. To do this, we will build polynomials of degrees 1 through 10. For each one, we will plot the resulting function, and compute the training error and test error.

Since we know that `mysty` and `myst` have a polynomial relationship, and given our layout of our data, we can actually just use linear regression in `sklearn` to output various polynomial relationships. Below is an example of this process for a 2-degree polynomial, ie $mysty = b_0 + b_1\cdot myst + b_2\cdot myst^2$.

We will divide and conquer this part. Slack me your code as you have it and I'll add it here. 

In [ ]:
# General x-values for fitting and plotting
x = np.linspace(np.min(train_data[:,1])-0.1, np.max(train_data[:,1])+0.1, 1000)
x = np.array([x]).T

x_mat = np.hstack([x,x**2,x**3,x**4,x**5,x**6,x**7,x**8,x**9,x**10])

In [ ]:
# Degree 1

In [ ]:
# Degree 2

# Create polynomial
lm2 = linear_model.LinearRegression()
model2 = lm2.fit(train_data[:,1:3],train_data[:,0])

# Create plot of training data with the polynomial 
plt.scatter(train_data[:,1],train_data[:,0])

preds2 = model2.predict(x_mat[:,:2])
plt.plot(x_mat[:,0],preds2, c="r")

# Compute the training error
train_preds2 = model2.predict(train_data[:,1:3])
train_error2 = compute_mse(train_preds2, train_data[:,0])

# Compute the testing error
test_preds2 = model2.predict(test_data[:,1:3])
test_error2 = compute_mse(test_preds2, test_data[:,0])

print("The training error is", round(train_error2,3), 
      " and the testing error is", round(test_error2,3))

In [ ]:
# Degree 3 

In [ ]:
# Degree 4 

In [ ]:
# Degree 5 

In [ ]:
# Degree 6

In [ ]:
# Degree 7

In [ ]:
# Degree 8 

In [ ]:
# Degree 9 

In [ ]:
# Degree 10 

#### What is the "right" one?

Based on these plots, which one is the "right" polynomial? 

#### Underfitting and Overfitting

There are two terms concerning fit that often used _underfitting_ and _overfitting_. Before we define those, let's plot the training errors and the testing errors both against the number of degrees. After the plot, write down a few things that you notice about this plot: 

In [ ]:
# Collate the training and testing errors 

train_errors = [???]
test_errors = [???]

In [ ]:
# Plotting training and testing errors

plt.plot(list(range(10)+1), train_errors, c="b")
plt.plot(list(range(10)+1), test_errors, c="r")

(Notes about the plot here!)
*  
*  
*  

**Underfitting** happens when we have not adequately fit our model to the training data. This happens when our training error is high. 

**Overfitting** happens when we have fit our model to perfectly to the training data and have made it less useful for making predictions to new data. This happens when the training error is low and the testing error is high. 

Underfitting and overfitting have to do with the _Bias/Variance trade off._ Our models are attempting to explain some generalization about our data and the underlying mechanics of the data. We want our model to be a good summary of the data in that it explains enough of the data's variations without being too sensitive to that particular data. 
* Every simplication of data will have _bias;_ the more simplistic the model, the more _bias_. 
* Every model also has _variance;_ the more complicated the model, the slightest change in the data will cause a huge change in the model. 

For an imperfect metaphor, say you want to buy a new blazer for work. You buy one in town. It's generic to a person of your approximate size and shape. There is _bias_ in creating this garmet to fit as many people as possible. You have the jacket tailored to your specific body. Now your friend wants to borrow it. You are both about the same size, but she says the jacket doesn't fit her. This is because it is overfit to you. In order for it to fit her, you will have to have it retailored and possibly many measurements will change. This is because the model of a jacket has high _variance._

Our goal in building models is to find a balance between bias and variance, which we exemplify as trying to minimize **both** training and testing errors. To achieve this balance, we want to have as much training and testing data as possible.

### Proportion of Train and Test

In our first exploration today of train and test, we used about 5% of the data for the training data and the rest for testing. Redo our experiments with 50% of the data in training and test. Then do them again for 95% of the data in the training set. Make the training and testing error plot and compare your results.

In [ ]:
# Redoing with 50% of data in training set

In [ ]:
# Redoing with 95% of data in training set

Now what degree of polynomial do you think is best? 

### Next time

Next time, we will continue exploring dividing our data into train and test and how the process of **cross-validation** helps us choose the "right" values for our models. 

### Final Thoughts

To finish up this lab, post your second two training and testing plots and state what you think the right balance of training and testing is. In other words, what percentage of your data should be in training? Justify your answer. Share your plots in a post on **#lab_submission** channel on slack with your answer. Your post must start with **Lab12** to get credit. 

If your have questions from this lab, post them to #lab_questions with the same preamble (i.e. starting with **Lab12**). If you have the same question, please use one of the emoji's to upvote the question. If you would like to answer someone's question, please use the thread function. This will tie your answer to their question. 

#### Resources consulted 

0. [Random selection in python](https://pynative.com/python-random-choice/)
1. [Setting random seed](https://pynative.com/python-random-seed/)
2. [Set difference](https://www.geeksforgeeks.org/python-set-difference/)